# CNN   small datasets 학습


- 딥러닝 모델은 대용량 데이터를 학습할 때 좋은 성능이 나온다. 그래서 학습할 Data가 적을 경우 좋은 성능을 내기가 어렵다.
  - 딥러닝은 **다양한 패턴의 많은 개수**의 데이터를 통해 feature engineering 과정 없이 원하는 문제를 해결하는 모델이다.
- Data가 많지 않아 학습에 어려움이 있을 때 사용 가능한 방법.
    1. **Data augmentation 활용**
        - 기존 데이터를 변경하여 데이터 개수를 늘린다.
        - 이미지의 경우 원본 이미지의 색깔, 각도 등을 약간씩 변형한 이미지들을 추가로 만들어 data의 수를 늘린다.
    2. **Pre-trained network의 활용**
        - 매우 큰 데이터셋으로 미리 Training한 모델의 파라미터(가중치)를 가져와서 풀려는 문제에 맞게 모델을 재보정해서 사용한다.
        - 성능이 좋은 학습된 모델을 사용하므로 적은 데이터에도 좋은 성능을 낼 수있다.

## Data for cats vs. dogs
- 2013년 Kaggle의 computer vision competition data 활용 https://www.kaggle.com/c/dogs-vs-cats/data
- 개와 고양이를 구분하기 위한 문제로 각 12,500개의 이미지를 포함
- Medium-resolution color JPEGs
- 25000장의 사진 중 4000장의 cats/dogs 사진(2000 cats, 2000 dogs) 만을 사용하여 학습하여 좋은 모형을 만들어 낼 수 있을까?
    - 학습: 2000, 검증: 1000, 테스트: 1000
    
![cats_vs_dogs_samples](https://s3.amazonaws.com/book.keras.io/img/ch5/cats_vs_dogs_samples.jpg)

##### 이미지 다운로드
- gdown 패키지 : 구글 드라이브의 공유파일 다운로드 패키지    
- `pip install gdown`
- 코랩에는 설치 되어 있음.

In [1]:
!pip install gdown --upgrade

                                              0.0/77.1 kB ? eta -:--:--
     -----                                    10.2/77.1 kB ? eta -:--:--
     ---------------------------------------- 77.1/77.1 kB 1.4 MB/s eta 0:00:00


In [2]:
import gdown
from zipfile import ZipFile # zip압축 풀기/하기
import os

def down_extract():
    url = 'https://drive.google.com/uc?id=1YIxDL0XJhhAMdScdRUfDgccAqyCw5-ZV'
    fname = 'cats_and_dogs_small.zip'   # 다운받아서 저장할 파일 이름.
    # 다운로드
    gdown.download(url, fname, quiet=False)
    # 압축풀기
    with ZipFile(fname) as zipFile: # 압축파일의 경로를 넣어서  ZipFile 객체 생성
        zipFile.extractall('data/cats_and_dogs_small')  # 압축 풀 디렉토리 넣어서 실행. / extractall: 전체압축풀기

In [3]:
down_extract()

Downloading...
From (uriginal): https://drive.google.com/uc?id=1YIxDL0XJhhAMdScdRUfDgccAqyCw5-ZV
From (redirected): https://drive.google.com/uc?id=1YIxDL0XJhhAMdScdRUfDgccAqyCw5-ZV&confirm=t&uuid=b0500c67-4c8a-4ee3-9749-12447c374a25
To: C:\Users\qkrtn\classes\02_DL\cats_and_dogs_small.zip
100%|█████████████████████████████████████████████████████████████████████████████| 90.8M/90.8M [00:02<00:00, 33.7MB/s]


# 하이퍼파라미터 정의

In [4]:
LEARNING_RATE = 0.001
N_EPOCH = 1
N_BATCH = 100 # 에폭당 스텝수를 보면 한스텝이 100개씩 돌아간것을 알 수 있다.

## 모델 구현 및 학습

- Input: $224 \times 224$ 픽셀의 RGB layer 
- Output: cat or dog (binary classification)  
- ImageDataGenerator를 이용해 파일시스템에 저장된 이미지 데이터셋을 학습시킨다.

In [5]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os
import numpy as np
import random 

random.seed(0)
np.random.seed(0)
tf.random.set_seed(0)

In [6]:
# 모델 생성
def get_model():
    model = keras.Sequential()

    model.add(layers.Conv2D(filters=64, kernel_size=3, padding='same', activation='relu', input_shape=(224, 224, 3)))
    model.add(layers.MaxPooling2D(padding='same'))

    model.add(layers.Conv2D(filters=64, kernel_size=3, padding='same', activation='relu'))
    model.add(layers.MaxPooling2D(padding='same'))

    model.add(layers.Conv2D(filters=128, kernel_size=3, padding='same', activation='relu'))
    model.add(layers.MaxPooling2D(padding='same'))

    model.add(layers.Conv2D(filters=128, kernel_size=3, padding='same', activation='relu'))
    model.add(layers.MaxPooling2D(padding='same'))

    model.add(layers.Conv2D(filters=256, kernel_size=3, padding='same', activation='relu'))
    model.add(layers.MaxPooling2D(padding='same'))

    model.add(layers.Conv2D(filters=256, kernel_size=3, padding='same', activation='relu'))
    model.add(layers.MaxPooling2D(padding='same'))

    model.add(layers.Flatten())
    
    # output layer: 이진분류 -> positive일 확률을 출력. unit=1, activation=sigmoid
    model.add(layers.Dense(units=1, activation='sigmoid', name='binary_classification_output'))

    model.compile(optimizer=optimizers.Adam(learning_rate=LEARNING_RATE), 
                  loss='binary_crossentropy', 
                  metrics=['accuracy'])
    return model


# ImageDataGenerator 생성

In [7]:
os.getcwd()

'C:\\Users\\qkrtn\\classes\\02_DL'

In [8]:
# 데이터셋 디렉토리 경로
trainset_dir = 'data/cats_and_dogs_small/train'
valset_dir = 'data/cats_and_dogs_small/validation/'
testset_dir = 'data/cats_and_dogs_small/test/'

In [9]:
# image augmentation 적용하지 않음.
train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)
val_datagen = ImageDataGenerator(rescale=1./255)

# Iterator 생성->raw data파일과 연결.
train_iter = train_datagen.flow_from_directory(trainset_dir, # 이미지데이터셋이 저장된 디렉토리 / 이거랑 연결해야해! 
                                               target_size=(224,224), # 이미지를 읽어올 때 사이즈가 다르므로 resize크기 정하는것.
                                               class_mode='binary',# output(y)의 처리방식-이진분류 / 다중분류할지 이진 분류할지 정하는것.
                                               batch_size=N_BATCH # 배치 크기. 한번에 몇개씩 읽어와서 모델에 주입할지.
                                               )

val_iter = val_datagen.flow_from_directory(valset_dir, target_size=(224, 224), class_mode='binary', batch_size=N_BATCH)
test_iter = test_datagen.flow_from_directory(testset_dir, target_size=(224, 224), class_mode='binary', batch_size=N_BATCH)

Found 2000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.


In [10]:
print("==class index-class name확인")
print(train_iter.class_indices)

==class index-class name확인
{'cats': 0, 'dogs': 1}


In [11]:
print('==에폭당 step 수')
print(len(train_iter), len(val_iter), len(test_iter))

==에폭당 step 수
20 10 10



##  Model Training(학습)

In [13]:
g_drive_path = '/content/drive/MyDrive/' # 구글 드라이브에서 저장할 때 사용하는 것.
save_dir_path = os.path.join(g_drive_path, 'SAVED_MODELS', 'cat_dog_model_no_aug')
os.makedirs(save_dir_path, exist_ok=True)

save_weight_path = os.path.join(save_dir_path, 'ckpt')

mc_callback = keras.callbacks.ModelCheckpoint(filepath=save_weight_path, 
                                              save_weights_only=True, # 파라미터(가중치)만 저장
                                              save_best_only=True,# 성능이 좋은 것
                                              monitor='val_loss',
                                              verbose=1)

es_callback = keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, verbose=1)

In [12]:
model = get_model()
hist = model.fit(train_iter, epochs=N_EPOCH,
                 steps_per_epoch=len(train_iter),
                 validation_data=val_iter, 
                 validation_steps=len(val_iter),
#                  callbacks=[mc_callback, es_callback]
                 )

20/20 [==============================] - 120s 6s/step - loss: 0.6935 - accuracy: 0.5160 - val_loss: 0.6907 - val_accuracy: 0.5190


In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 6))
plt.plot(hist.epoch, hist.history['loss'], label='train')
plt.plot(hist.epoch, hist.history['val_loss'], label='validation')
plt.legend()
plt.show()

##### 저장된 모델 조회

In [ ]:
# 모델 생성
saved_model1 = get_model()
saved_model1.evaluate(test_iter)

In [ ]:
# 새로 생성된 모델에 저장 best wegith 덮어씌우기
saved_model1.load_weights(save_weight_path)

saved_model1.evaluate(test_iter)

# 새로운 데이터 추론

In [ ]:
from tensorflow.keras.preprocessing.image import load_img, img_to_array

def load_preprocessing(image_path):
    """
    경로의 이미지를 읽어서 전처리(scaling) 한 뒤 반환.
    [Parameter]
       image_path: str - 변환할 이미지 경로(Path)
    [Return]
       ndarray: 전처리한 이미지
    """
    raw_img = load_img(image_path, target_size=(224,224))
    img_array = img_to_array(raw_img)
    img_array = img_array[np.newaxis, ...]
    img_array = img_array.astype('float32')/255
    return img_array

In [ ]:
new_img_path = 'dog.jpg'
new_X = load_preprocessing(new_img_path)

pred = saved_model1.predict(new_X)
print(pred)
print(np.where(pred>=0.5, "개", "고양이"))

# Image data augmentation 사용

- 학습 이미지의 수가 적어서 overfitting이 발생할 가능성을 줄이기 위해 기존 훈련 데이터로부터 그럴듯하게 이미지 변환을 통해서 이미지(데이터)를 늘리는 작업을 Image augmentation
- train_set에만 적용, validation, test set에는 적용하지 않는다. (rescaling만 한다.)

##### 하이퍼파라미터 정의

##### ImageDataGenerator 생성

In [14]:
# 데이터셋 디렉토리 경로
trainset_dir = 'data/cats_and_dogs_small/train'
valset_dir = 'data/cats_and_dogs_small/validation/'
testset_dir = 'data/cats_and_dogs_small/test/'

In [15]:
# Image Augmentation 적용. ==> Train set에만 적용.
train_datagen2 = ImageDataGenerator(rescale=1./255, 
                                    rotation_range=40, 
                                    width_shift_range=0.2, 
                                    height_shift_range=0.2, 
                                    zoom_range=0.2, 
                                    horizontal_flip=True, 
                                    fill_mode='constant')

val_datagen2 = ImageDataGenerator(rescale=1./255)
test_datagen2 = ImageDataGenerator(rescale=1./255)

In [16]:
train_iter2 = train_datagen2.flow_from_directory(trainset_dir, target_size=(224,224), class_mode='binary', 
                                                 batch_size=N_BATCH)

val_iter2 = val_datagen2.flow_from_directory(valset_dir, target_size=(224,224), class_mode='binary', batch_size=N_BATCH)

test_iter2 = test_datagen2.flow_from_directory(testset_dir, target_size=(224,224), class_mode='binary', batch_size=N_BATCH)

Found 2000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.


##### 모델 생성, 컴파일, 학습

In [20]:
# # g_drive_path = '/content/drive/MyDrive'
# save_dir_path = os.path.join(g_drive_path, "saved_model", "cat_dog_model_aug")
# mc_callback2 = keras.callbacks.ModelCheckpoint(save_dir_path, save_best_only=True, monitor='val_loss', verbose=1)
# es_callback2 = keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, verbose=1)


model2 = get_model()

hist2 = model2.fit(train_iter2, epochs=N_EPOCH, 
                   steps_per_epoch=len(train_iter2), # 학습시 1에폭당 step수 => 1에폭이 언제 끝났는지 알려준다.
                   validation_data=val_iter2, 
                   validation_steps=len(val_iter2),) # 검증시 1에폭당 step수 
#                    callbacks=[mc_callback2, es_callback2])

20/20 [==============================] - 128s 6s/step - loss: 0.6949 - accuracy: 0.5040 - val_loss: 0.6933 - val_accuracy: 0.5000


In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 6))
plt.plot(hist2.epoch, hist2.history['loss'], label='train')
plt.plot(hist2.epoch, hist2.history['val_loss'], label='validation')
plt.legend()
plt.show()

##### 저장된 모델 Loading 후 평가(사용)

In [ ]:
from tensorflow.keras import models
from tensorflow.keras.preprocessing.image import load_img, img_to_array

saved_model2 = models.load_model(save_dir_path)

##### 새로운데이터 추론

In [ ]:
import numpy as np
def load_preprocessing_image(path):
    raw_img = load_img(path, target_size=(224, 224))  #이미지 읽어오기
    img_array = img_to_array(raw_img)
    img_array = img_array[np.newaxis, ...]
    img_array = img_array.astype('float32')/255
    return img_array

In [ ]:
img_path = 'dog.jpg'
data = load_preprocessing_image(img_path)
# data.shape
pred = saved_model2.predict(data)
pred_label = np.where(pred[0]>=0.5, "개", "고양이")
print(pred[0], pred_label)
